In [ ]:
from pprint import pprint
from pathlib import Path

import duckdb
import ipywidgets as widgets
import json

from mosaic_widget import MosaicWidget

In [ ]:
# Optional: enable logging in Mosaic widget
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
# Change working directory to mosaic root
%cd ../..

In [ ]:
con = duckdb.connect()
con.execute("LOAD httpfs");

In [ ]:
mosaic = MosaicWidget(con=con)

In [ ]:
def get_specs(folder_path):
    folder = Path(folder_path)
    for file_path in sorted(folder.glob("*.json")):
        data = json.loads(file_path.read_text())
        meta = data.get("meta")
        if meta:
            yield (meta.get("title"), file_path)

In [ ]:
weather = Path("specs/json/weather.json")

dropdown = widgets.Dropdown(
    options=get_specs("specs/json"),
    value=weather,
    description="Example:",
)


def on_change(change):
    open_spec(change["new"])


def open_spec(spec):
    mosaic.spec = json.loads(spec.read_text())


dropdown.observe(on_change, "value")

open_spec(weather)

output = widgets.Output()


@output.capture(clear_output=True)
def handle_change(change):
    pprint(change.new)


mosaic.observe(handle_change, names=["params"])

In [ ]:
widgets.VBox([dropdown, mosaic, output])